In [1]:
import time
import datetime

import cst_python as cst
import toolpy as tp
from toolpy.integrations import groq

import sys 
sys.path.append('../src')

import gbyg as gg

c:\Users\weslley.santos\Documents\GDataByGAgents\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [3]:
groq_interface = groq.GroqInterface(model=groq.GroqModel.LLAMA3_70B, n_retry=5)

registry = tp.llm.LLMRegistry()
registry.registry(model_name="llama3-70b", interface=groq_interface, default=True)

In [4]:
mind = cst.Mind()

gg.agent.agent_constructor(mind)

for codelet in mind.code_rack.all_codelets:
    codelet.time_step = 100

memories_input = mind.raw_memory.get_all_of_type("MemoriesInput")[0]
memory_stream = mind.raw_memory.get_all_of_type("MemoryStream")[0]
query_memory = mind.raw_memory.get_all_of_type("QueryMemory")[0]
agent_time = mind.raw_memory.get_all_of_type("AgentTime")[0]
agent_info = mind.raw_memory.get_all_of_type("AgentInfo")[0]
agent_summary_description = mind.raw_memory.get_all_of_type("AgentSummaryDescription")[0]

retrieved_memories = mind.raw_memory.get_all_of_type("RetrievedMemories")[0]

##########################################################################################

generated_questions = mind.raw_memory.get_all_of_type("GeneratedQuestions")[0]
generated_insights = mind.raw_memory.get_all_of_type("GeneratedInsights")[0]
statements = mind.raw_memory.get_all_of_type("Statements")[0]

c:\Users\weslley.santos\Documents\GDataByGAgents\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
agent_info.set_info({"name":"Alex", "Age":25, "Traits":"friendly, outgoing, hospitable"})

-1

In [6]:
mind.start()

In [7]:
input_memories  = [
  {
    "index": "0",
    "type": "event",
    "created": "2024-06-01 08:00:00",
    "description": "Alex attended a nature photography workshop in the city park.",
  },
  {
    "index": "1",
    "type": "thought",
    "created": "2024-06-01 19:00:00",
    "description": "Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.",
  },
  {
    "index": "2",
    "type": "event",
    "created": "2024-06-05 10:00:00",
    "description": "Alex completed a major software development project at work, delivering the final product to the client.",
  },
  {
    "index": "3",
    "type": "thought",
    "created": "2024-06-05 20:00:00",
    "description": "After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.",
  },
  {
    "index": "4",
    "type": "event",
    "created": "2024-06-10 16:00:00",
    "description": "Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.",
  }
]

for m in input_memories:
    m["last_acessed"] = datetime.datetime.fromisoformat(m["created"]).timestamp()

In [8]:
input_memories

[{'index': '0',
  'type': 'event',
  'created': '2024-06-01 08:00:00',
  'description': 'Alex attended a nature photography workshop in the city park.',
  'last_acessed': 1717239600.0},
 {'index': '1',
  'type': 'thought',
  'created': '2024-06-01 19:00:00',
  'description': 'Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.',
  'last_acessed': 1717279200.0},
 {'index': '2',
  'type': 'event',
  'created': '2024-06-05 10:00:00',
  'description': 'Alex completed a major software development project at work, delivering the final product to the client.',
  'last_acessed': 1717592400.0},
 {'index': '3',
  'type': 'thought',
  'created': '2024-06-05 20:00:00',
  'description': 'After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
  'last_acessed': 1717628400.0},
 {'index': '4',
  'type': 'event',
  'created': '2024-06-10 16:00:00',
  'description': 'Alex went on a

In [9]:
memories_input_queue = memories_input.get_info()
for memory in input_memories:
    memories_input_queue.append(memory)

In [10]:
memories_input_queue

deque([{'index': '0',
        'type': 'event',
        'created': '2024-06-01 08:00:00',
        'description': 'Alex attended a nature photography workshop in the city park.',
        'last_acessed': 1717239600.0},
       {'index': '1',
        'type': 'thought',
        'created': '2024-06-01 19:00:00',
        'description': 'Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.',
        'last_acessed': 1717279200.0},
       {'index': '2',
        'type': 'event',
        'created': '2024-06-05 10:00:00',
        'description': 'Alex completed a major software development project at work, delivering the final product to the client.',
        'last_acessed': 1717592400.0},
       {'index': '3',
        'type': 'thought',
        'created': '2024-06-05 20:00:00',
        'description': 'After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
        'last_acessed'

In [18]:
time.sleep(10)

len(memory_stream.get_info())

5

In [19]:
questions = generated_questions.get_info()
for q in questions:
    print(q)

What are Alex's hobbies or interests?
What are Alex's accomplishments or achievements?
What are the different types of activities Alex engages in?


In [20]:
retrieved_mem = []
for q in questions:
    query_memory.set_info(q)
    time.sleep(2)

    retrieved_memories_list = retrieved_memories.get_info()
    for m in retrieved_memories_list:
        retrieved_mem.append(m['description'])

retrieved_mem

['After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
 'Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.',
 'After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
 'Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.',
 'After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
 'Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.']

In [21]:
retrieved_mem = list(set(retrieved_mem))

In [22]:
retrieved_mem

['After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.',
 'Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.']

In [23]:
statements.set_info(retrieved_mem)
time.sleep(10)  

insights = generated_insights.get_info()
print("Generated Insights:")
for insight in insights:
    print(insight)

Generated Insights:
Alex is goal-oriented and values a sense of accomplishment (because of 1)
Alex has an adventurous spirit and enjoys exploring nature (because of 2)
Alex is capable of overcoming challenges (because of 1)
Alex appreciates taking breaks and engaging in leisure activities (because of 2)
Alex has an interest in photography (because of 2)


In [24]:
agent_time.set_info(gg.time_utils.TimeInterval.ONE_DAY)
time.sleep(5)
agent_summary_description.get_info()

"{'name': 'Alex', 'Age': 25, 'Traits': 'friendly, outgoing, hospitable'}\nAlex is an adventurous and accomplished individual who appreciates nature and takes pride in overcoming challenges.\nAlex is likely on a break or vacation after completing a project, and is currently engaged in leisure activities such as travel and photography.\n"

In [25]:
print(agent_summary_description.get_info())

{'name': 'Alex', 'Age': 25, 'Traits': 'friendly, outgoing, hospitable'}
Alex is an adventurous and accomplished individual who appreciates nature and takes pride in overcoming challenges.
Alex is likely on a break or vacation after completing a project, and is currently engaged in leisure activities such as travel and photography.



In [ ]:
importance_prompt = '''On the scale of 1 to 10, where 1 is purely mundane 
(e.g., brushing teeth, making bed) and 10 is extremely poignant 
(e.g., a break up, college acceptance), rate the likely poignancy of 
the following piece of memory.

Memory: {memory}'''